<div style="font-size: 200%; font-weight: bold; color: maroon;">Machine Learning en SparkR - datos distribuidos</div>
<!-- v. 1.1 - Paulo Villegas, 2017 -->

Adaptado de un [ejemplo de SparkR](https://github.com/apache/spark/blob/master/examples/src/main/r/ml/ml.R).

Vamos a ajustar un GLM para hacer una regresión de una variable contra dos predictores, uno numérico y otro categórico.

In [ ]:
# Load SparkR and create a session
library(SparkR)
spark <- sparkR.session( "local[*]" );

# Carga de datos

Usamos el típico [dataset de IRIS](https://en.wikipedia.org/wiki/Iris_flower_data_set). Está disponible por defecto en R; para trabajar con él en Spark lo convertimos en `SparkDataFrame`

In [ ]:
df <- as.DataFrame(iris)

head(df,5)

Separamos en conjunto de entrenamiento y test

In [ ]:
df.split = randomSplit( df, c(80,20))

In [ ]:
df.split

# Regresión

Vamos a realizar una regresión: intentamos calcular una variable (longitud de sépalo) a partir de dos:
* anchura de sépalo (variable numérica)
* especie de flor (variable categórica)

## Versión sintaxis específica de Spark

Usamos [`spark.glm`](https://spark.apache.org/docs/latest/api/R/spark.glm.html)

In [ ]:
# Fit a gaussian GLM model over the dataset.  
# This model uses 2 base features, one numerical, the other categorical
gaussianGLM <- spark.glm(df.split[[1]], 
                         Sepal_Length ~ Sepal_Width + Species,
                         family = "gaussian")

In [ ]:
summary(gaussianGLM)

In [ ]:
df.pred <- predict( gaussianGLM, df.split[[2]])

In [ ]:
collect( sample(df.pred,FALSE,0.2) )

In [ ]:
library('ggplot2')
qplot( label, prediction, data=collect(df.pred) ) + coord_fixed() + geom_smooth( method="lm")

## Versión con sintaxis nativa de R

Usamos el método "oficial" de R: [`glm`](https://spark.apache.org/docs/latest/api/R/glm.html) (que internamente llama a `spark.glm` cuando el objeto pasado en el argumento `data` es un `SparkDataFrame`)

In [ ]:
# Fit a gaussian GLM model over the dataset.  This model uses 2 original (base) features.
gaussianGLM2 <- glm(Sepal_Length ~ Sepal_Width + Species, 
                    data = df.split[[1]], 
                    family = "gaussian")

In [ ]:
summary(gaussianGLM2)

----------------------

# Ejercicio

## Interacción entre características de entrada

Hemos estimado la longitud de los sépalos en función de su anchura y de la especie de planta, pero considerando ambas características (_Sepal_Width_ y _Species_ ) de forma separada. Sin embargo, es razonable esperar que ambas estén relacionadas: la especie influye en la anchura del pétalo (además de en su longitud). Por lo tanto, desde el punto de vista de la regresión, puede ser interesante considerar un término más que tenga en cuenta las _interacciones_ entre esas dos características.

Las fórmulas de R permiten definir esas interacciones fácilmente; SparkR soporta usar términos de interacción. 
Puede verse la sintaxis necesaria:
* en la [documentación sobre glm](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/glm.html). 
* en la [documentación sobre fórmulas en R](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/formula.html)


<span style="color: red;">Ejercicio:</span> *estime la longitud de los sépalos en función de 3 variables:* 
* `Sepal_Width`
* `Species` 
* y la interacción entre `Sepal_Width` y `Species`

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>
